In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, recall_score
from sklearn.preprocessing import StandardScaler

def repeated_k_fold(model, X, y, n_splits=5, n_repeats=10):
    """Führt repeated k-fold cross-validation durch und berechnet die Metriken."""
    # Repeated Stratified K-Fold
    rkf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

    # Metriken definieren
    accuracy_train = []
    accuracy_test = []
    f1 = []
    recall = []  
    roc_auc = []

    for train_index, test_index in rkf.split(X, y):
        # Splitte die Daten in Trainings- und Testdaten
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Trainiere das Modell
        model.fit(X_train, y_train)

        # Vorhersagen für Trainings- und Testdaten
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Berechne die Metriken
        accuracy_train.append(accuracy_score(y_train, y_train_pred))
        accuracy_test.append(accuracy_score(y_test, y_test_pred))
        f1.append(f1_score(y_test, y_test_pred))
        recall.append(recall_score(y_test, y_test_pred))  
        roc_auc.append(roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))

    # Durchschnittswerte berechnen
    return {
        "Train Accuracy": np.mean(accuracy_train),
        "Test Accuracy": np.mean(accuracy_test),
        "F1-Score": np.mean(f1),
        "Recall": np.mean(recall), 
        "ROC-AUC": np.mean(roc_auc),
    }

# Pfad zur Datei
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Vergleich_links_rechts_INV_UNINV\Basistabelle.xlsx"

# Excel-Datei einlesen
try:
    # Direkt die Datei einlesen (erstes Tabellenblatt standardmäßig)
    df = pd.read_excel(file_path)

    # Zielvariable (y) und Features (X) extrahieren
    y = df['Verletzungsstatus']
    X = df.drop(columns=['Verletzungsstatus'])  # Entferne die Zielvariable aus den Features

    # Skalierung der Features (X) mit StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)  # Normiert X auf den Standardbereich (Mittelwert = 0, Std = 1)

    # Modelle definieren
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42),
        "LightGBM": LGBMClassifier(verbose=-1,random_state=42),
        "SVC": SVC(probability=True, random_state=42),
        "k-Nearest Neighbors": KNeighborsClassifier(),
        "MLP Classifier": MLPClassifier(max_iter=1000, random_state=42),
        "Gaussian Naive Bayes": GaussianNB(),
        "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),
        "Quadratic Discriminant Analysis": QuadraticDiscriminantAnalysis(),
        "Bagging Classifier": BaggingClassifier(random_state=42),
        "Extra Trees": ExtraTreesClassifier(random_state=42),
    }

    # Ergebnisse speichern
    results = []

    # Validierung jedes Modells
    for model_name, model in models.items():
        print(f"Modell wird validiert: {model_name}")
        metrics = repeated_k_fold(model, X_scaled, y)  # Verwende X_scaled statt X
        results.append({"Model": model_name, **metrics})

    # Ergebnisse in DataFrame konvertieren und sortieren
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by="ROC-AUC", ascending=False)

    # Ergebnisse anzeigen
    print("\nErgebnisse der Modelle:")
    print(results_df)

except FileNotFoundError:
    print("Die Datei wurde nicht gefunden. Bitte überprüfen Sie den Pfad.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")


Modell wird validiert: Logistic Regression
Modell wird validiert: Decision Tree
Modell wird validiert: Random Forest
Modell wird validiert: Gradient Boosting
Modell wird validiert: XGBoost
Modell wird validiert: LightGBM
Modell wird validiert: SVC
Modell wird validiert: k-Nearest Neighbors
Modell wird validiert: MLP Classifier
Modell wird validiert: Gaussian Naive Bayes
Modell wird validiert: Linear Discriminant Analysis
Modell wird validiert: Quadratic Discriminant Analysis


C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Varia

Modell wird validiert: Bagging Classifier
Modell wird validiert: Extra Trees

Ergebnisse der Modelle:
                              Model  Train Accuracy  Test Accuracy  F1-Score  \
3                 Gradient Boosting        1.000000       0.797500  0.811820   
0               Logistic Regression        0.992093       0.775435  0.790945   
4                           XGBoost        1.000000       0.778080  0.795558   
5                          LightGBM        1.000000       0.777355  0.796665   
13                      Extra Trees        1.000000       0.805072  0.832207   
6                               SVC        0.945941       0.771920  0.805519   
2                     Random Forest        1.000000       0.809674  0.833884   
12               Bagging Classifier        0.983773       0.765942  0.772612   
8                    MLP Classifier        1.000000       0.755797  0.781937   
9              Gaussian Naive Bayes        0.838044       0.742065  0.772638   
7               k-

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, recall_score
from sklearn.preprocessing import StandardScaler

def repeated_k_fold(model, X, y, n_splits=5, n_repeats=10):
    """Führt repeated k-fold cross-validation durch und berechnet die Metriken."""
    # Repeated Stratified K-Fold
    rkf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

    # Metriken definieren
    accuracy_train = []
    accuracy_test = []
    f1 = []
    recall = []  
    roc_auc = []

    for train_index, test_index in rkf.split(X, y):
        # Splitte die Daten in Trainings- und Testdaten
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Trainiere das Modell
        model.fit(X_train, y_train)

        # Vorhersagen für Trainings- und Testdaten
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Berechne die Metriken
        accuracy_train.append(accuracy_score(y_train, y_train_pred))
        accuracy_test.append(accuracy_score(y_test, y_test_pred))
        f1.append(f1_score(y_test, y_test_pred))
        recall.append(recall_score(y_test, y_test_pred))  
        roc_auc.append(roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))

    # Durchschnittswerte berechnen
    return {
        "Train Accuracy": np.mean(accuracy_train),
        "Test Accuracy": np.mean(accuracy_test),
        "F1-Score": np.mean(f1),
        "Recall": np.mean(recall), 
        "ROC-AUC": np.mean(roc_auc),
    }

# Pfad zur Datei
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Vergleich_links_rechts_INV_UNINV\sortierte_Basistabelle.xlsx"

# Excel-Datei einlesen
try:
    # Direkt die Datei einlesen (erstes Tabellenblatt standardmäßig)
    df = pd.read_excel(file_path)

    # Zielvariable (y) und Features (X) extrahieren
    y = df['Verletzungsstatus']
    X = df.drop(columns=['Verletzungsstatus'])  # Entferne die Zielvariable aus den Features

    # Skalierung der Features (X) mit StandardScaler
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)  # Normiert X auf den Standardbereich (Mittelwert = 0, Std = 1)

    # Modelle definieren
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42),
        "XGBoost": XGBClassifier(eval_metric="logloss", random_state=42),
        "LightGBM": LGBMClassifier(verbose=-1,random_state=42),
        "SVC": SVC(probability=True, random_state=42),
        "k-Nearest Neighbors": KNeighborsClassifier(),
        "MLP Classifier": MLPClassifier(max_iter=1000, random_state=42),
        "Gaussian Naive Bayes": GaussianNB(),
        "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),
        "Quadratic Discriminant Analysis": QuadraticDiscriminantAnalysis(),
        "Bagging Classifier": BaggingClassifier(random_state=42),
        "Extra Trees": ExtraTreesClassifier(random_state=42),
    }

    # Ergebnisse speichern
    results = []

    # Validierung jedes Modells
    for model_name, model in models.items():
        print(f"Modell wird validiert: {model_name}")
        metrics = repeated_k_fold(model, X_scaled, y)  # Verwende X_scaled statt X
        results.append({"Model": model_name, **metrics})

    # Ergebnisse in DataFrame konvertieren und sortieren
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by="ROC-AUC", ascending=False)

    # Ergebnisse anzeigen
    print("\nErgebnisse der Modelle:")
    print(results_df)

except FileNotFoundError:
    print("Die Datei wurde nicht gefunden. Bitte überprüfen Sie den Pfad.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")


Modell wird validiert: Logistic Regression
Modell wird validiert: Decision Tree
Modell wird validiert: Random Forest
Modell wird validiert: Gradient Boosting
Modell wird validiert: XGBoost
Modell wird validiert: LightGBM
Modell wird validiert: SVC
Modell wird validiert: k-Nearest Neighbors
Modell wird validiert: MLP Classifier
Modell wird validiert: Gaussian Naive Bayes
Modell wird validiert: Linear Discriminant Analysis
Modell wird validiert: Quadratic Discriminant Analysis


C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\boehmer\AppData\Local\anaconda3\Lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Varia

Modell wird validiert: Bagging Classifier
Modell wird validiert: Extra Trees

Ergebnisse der Modelle:
                              Model  Train Accuracy  Test Accuracy  F1-Score  \
0               Logistic Regression        1.000000       0.843659  0.851363   
4                           XGBoost        1.000000       0.846413  0.860532   
6                               SVC        0.966683       0.828841  0.852547   
8                    MLP Classifier        1.000000       0.843659  0.857925   
3                 Gradient Boosting        1.000000       0.848913  0.861590   
2                     Random Forest        1.000000       0.829239  0.849319   
5                          LightGBM        1.000000       0.814674  0.831586   
13                      Extra Trees        1.000000       0.821232  0.842617   
12               Bagging Classifier        0.987392       0.808587  0.813092   
9              Gaussian Naive Bayes        0.866255       0.787138  0.811697   
7               k-

In [35]:
import pandas as pd

# Datei einlesen
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\sortierte_Basistabelle.xlsx"
df = pd.read_excel(file_path)

# Dummy-Variablen für die "Sex"-Spalte erstellen (Spalte 5)
sex_dummies = pd.get_dummies(df['Sex'], prefix='Sex', drop_first=False)

# Füge die Dummy-Variablen dem DataFrame hinzu
df = pd.concat([df, sex_dummies], axis=1)

# Speichern der neuen Tabelle mit den Dummy-Spalten
output_file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\sortierte_Basistabelle_mit_Sex_Dummies.xlsx"
df.to_excel(output_file_path, index=False)

print(f"Die Dummy-Variablen wurden hinzugefügt und die Datei wurde gespeichert unter: {output_file_path}")


Die Dummy-Variablen wurden hinzugefügt und die Datei wurde gespeichert unter: K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\sortierte_Basistabelle_mit_Sex_Dummies.xlsx


In [37]:
import pandas as pd

# Datei einlesen
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\sortierte_Basistabelle.xlsx"
df = pd.read_excel(file_path)
df

,Type,Code,Name,Surname,Sex,Birthdate,Age,Weight,Height,Injured Leg,...,INV_ISO_Unterschied Extension Flexion,UNINV_ISO_Unterschied Extension Flexion,Zeit Post-OP,Geb.-Datum,Diagnose,Begleitverletzungen,betroffene Seite,OP-Tag,Sportart,Unnamed: 122
0,P,53,Romina,Bell,female,1993-05-14,31.0,63.0,1.63,L,...,0.352857,0.912698,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
1,P,61,Dorian,Zehentmayr,male,2005-03-03,19.0,63.0,1.77,L,...,0.870635,1.211587,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
2,P,141,Julian,Schuetter,male,1998-03-14,26.0,93.0,1.85,L,...,0.786237,1.302903,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
3,P,179,Leon,Mayr,male,2005-05-05,19.0,72.0,1.85,R,...,0.859722,0.288611,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
4,P,188,Harry,Laidlaw,male,1996-03-01,28.0,82.0,1.79,R,...,0.826098,1.376341,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,N,N24_041,Alexander,Schwab,male,2001-02-16,23.0,78.0,1.82,NaN,...,1.121795,1.323077,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
113,N,546,Philipp,Viertler,male,2000-11-13,23.0,75.0,1.83,NaN,...,1.293067,0.841600,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
114,N,581,Johannes,Wagner,male,2000-04-24,24.0,82.0,1.90,NaN,...,1.310000,1.284268,NaN,2002-08-07,NaN,NaN,NaN,NaT,Volleyball,NaN
115,N,535,Tobias,Wagner,male,2002-08-17,21.0,75.0,1.93,NaN,...,1.174800,1.332133,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN


In [39]:
import pandas as pd

# Datei einlesen
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\sortierte_Basistabelle.xlsx"
df = pd.read_excel(file_path)

# Erstellen der Dummy-Variablen für die 'Sex'-Spalte
sex_dummies = pd.get_dummies(df['Sex'], prefix='Geschlecht', drop_first=False)

# Ändere die Spaltennamen, um die gewünschte Bezeichnung zu erhalten
sex_dummies.columns = ['Geschlecht_weiblich', 'Geschlecht_männlich']

# Füge die Dummy-Spalten zum DataFrame hinzu
df = pd.concat([df, sex_dummies], axis=1)

# Anzeigen des DataFrames, um die neuen Spalten zu überprüfen
df.head()


,Type,Code,Name,Surname,Sex,Birthdate,Age,Weight,Height,Injured Leg,...,Zeit Post-OP,Geb.-Datum,Diagnose,Begleitverletzungen,betroffene Seite,OP-Tag,Sportart,Unnamed: 122,Geschlecht_weiblich,Geschlecht_männlich
0,P,53,Romina,Bell,female,1993-05-14,31.0,63.0,1.63,L,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,True,False
1,P,61,Dorian,Zehentmayr,male,2005-03-03,19.0,63.0,1.77,L,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,False,True
2,P,141,Julian,Schuetter,male,1998-03-14,26.0,93.0,1.85,L,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,False,True
3,P,179,Leon,Mayr,male,2005-05-05,19.0,72.0,1.85,R,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,False,True
4,P,188,Harry,Laidlaw,male,1996-03-01,28.0,82.0,1.79,R,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,False,True


In [41]:
import pandas as pd

# Datei einlesen
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\sortierte_Basistabelle.xlsx"
df = pd.read_excel(file_path)

# Erstellen der Dummy-Variablen für die 'Sex'-Spalte
sex_dummies = pd.get_dummies(df['Sex'], prefix='Geschlecht', drop_first=False)

# Umwandeln der Dummy-Werte von True/False zu 1/0
sex_dummies = sex_dummies.astype(int)

# Ändere die Spaltennamen, um die gewünschte Bezeichnung zu erhalten
sex_dummies.columns = ['Geschlecht_weiblich', 'Geschlecht_männlich']

# Füge die Dummy-Spalten zum DataFrame hinzu
df = pd.concat([df, sex_dummies], axis=1)

# Anzeigen des DataFrames, um die neuen Spalten zu überprüfen
df.head()


,Type,Code,Name,Surname,Sex,Birthdate,Age,Weight,Height,Injured Leg,...,Zeit Post-OP,Geb.-Datum,Diagnose,Begleitverletzungen,betroffene Seite,OP-Tag,Sportart,Unnamed: 122,Geschlecht_weiblich,Geschlecht_männlich
0,P,53,Romina,Bell,female,1993-05-14,31.0,63.0,1.63,L,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,1,0
1,P,61,Dorian,Zehentmayr,male,2005-03-03,19.0,63.0,1.77,L,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,0,1
2,P,141,Julian,Schuetter,male,1998-03-14,26.0,93.0,1.85,L,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,0,1
3,P,179,Leon,Mayr,male,2005-05-05,19.0,72.0,1.85,R,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,0,1
4,P,188,Harry,Laidlaw,male,1996-03-01,28.0,82.0,1.79,R,...,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,0,1


In [43]:
# Speicherpfad definieren
output_file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\sortierte_Basistabelle_geschlecht.xlsx"

# DataFrame als Excel-Datei speichern
df.to_excel(output_file_path, index=False)

print(f"Die Datei wurde erfolgreich gespeichert unter: {output_file_path}")


Die Datei wurde erfolgreich gespeichert unter: K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\sortierte_Basistabelle_geschlecht.xlsx


In [9]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Pfad zur Excel-Datei
file_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Work_ergänzen\datum_messung.xlsx"

# Excel-Datei einlesen
try:
    # Excel-Datei einlesen
    df = pd.read_excel(file_path)

    # Überprüfen, ob die Spalten 5 und 6 existieren
    if df.shape[1] > 5:
        # Laden der Excel-Datei mit openpyxl für die Zellformatierung
        wb = load_workbook(file_path)
        sheet = wb.active

        # Definieren des gelben Füllformats für die Zellen
        yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

        # Iteration über die Zeilen
        for row_idx in range(2, sheet.max_row + 1):  # Ab Zeile 2 (wegen Header)
            cell_5 = sheet.cell(row=row_idx, column=5)  # Spalte 5 (E)
            cell_6 = sheet.cell(row=row_idx, column=6)  # Spalte 6 (F)

            # Vergleiche die Werte in den beiden Zellen
            if cell_5.value != cell_6.value:
                # Wenn die Werte unterschiedlich sind, färbe beide Zellen gelb
                cell_5.fill = yellow_fill
                cell_6.fill = yellow_fill

        # Speichern der Änderungen
        wb.save(file_path)
        print("Die Zellen wurden entsprechend gefärbt.")
    else:
        print("Die Excel-Datei hat nicht genügend Spalten.")
    
except FileNotFoundError:
    print("Die Datei wurde nicht gefunden. Bitte überprüfen Sie den Pfad.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")


Die Zellen wurden entsprechend gefärbt.


In [11]:
import pandas as pd

# Pfade zur Haupttabelle und Zusatz-Tabelle
haupttabelle = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Work_ergänzen\sortierte_Basistabelle_geschlecht_datum.xlsx"
zusatz = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Work_ergänzen\datum_messung.xlsx"

try:
    # Laden der Tabellen
    df_haupt = pd.read_excel(haupttabelle)
    df_zusatz = pd.read_excel(zusatz)

    # Umbenennen der Spalten für bessere Übersicht
    df_haupt = df_haupt.rename(columns={df_haupt.columns[1]: 'Code'})  # Spalte 2 in Haupttabelle
    df_zusatz = df_zusatz.rename(columns={df_zusatz.columns[0]: 'Code', df_zusatz.columns[4]: 'Spalte_5'})  # Spalte 1 als Code, Spalte 5 als relevante Spalte

    # Mergen der Tabellen basierend auf der Code-Spalte
    merged_df = pd.merge(df_haupt, df_zusatz[['Code', 'Spalte_5']], on='Code', how='left')

    # Speichern der gemergten Tabelle
    merged_df.to_excel(r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Work_ergänzen\gemergte_tabelle.xlsx", index=False)
    print("Die Tabellen wurden erfolgreich gemergt und gespeichert.")

except FileNotFoundError:
    print("Eine der Dateien wurde nicht gefunden. Bitte überprüfen Sie den Pfad.")
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")


Die Tabellen wurden erfolgreich gemergt und gespeichert.


In [5]:
import os
import pandas as pd
from openpyxl import load_workbook

# Pfad zum Ordner mit den Excel-Dateien
folder_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Work_ergänzen"

# Ergebnisse speichern
results = []

# Alle Excel-Dateien im Ordner durchlaufen
for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        try:
            # Excel-Datei laden
            workbook = load_workbook(file_path, read_only=True)
            matching_sheets = []

            # Überprüfung aller Arbeitsblätter
            for sheet_name in workbook.sheetnames:
                sheet = workbook[sheet_name]
                if any(cell and cell[0] == "Isokinetisch Kon/Kon" for cell in sheet.iter_rows(min_col=1, max_col=1, values_only=True)):
                    matching_sheets.append(sheet_name)

            if len(matching_sheets) > 1:
                print(f"Mehrere passende Arbeitsblätter in Datei: {file}. Datei wird übersprungen.")
                continue
            elif len(matching_sheets) == 0:
                print(f"Kein passendes Arbeitsblatt in Datei: {file}. Datei wird übersprungen.")
                continue

            sheet = workbook[matching_sheets[0]]
            arbeit_extension_rechts = arbeit_extension_links = None
            arbeit_flexion_rechts = arbeit_flexion_links = None

            for row_idx, row in enumerate(sheet.iter_rows(min_col=1, max_col=1, values_only=True), start=1):
                if row and row[0] == "Arbeit pro Wiederholung (Newton-Meter - Beste Wiederholung)":
                    rows = list(sheet.iter_rows(min_row=row_idx, max_row=row_idx + 3, values_only=True))

                    for sub_row in rows:
                        if sub_row and len(sub_row) > 1:
                            if sub_row[0] == "Rechts":
                                arbeit_extension_rechts = sub_row[1]  # Eine Zelle nach rechts
                                arbeit_flexion_rechts = sub_row[4] if len(sub_row) > 4 else None  # Drei Zellen nach rechts
                            elif sub_row[0] == "Links":
                                arbeit_extension_links = sub_row[1]  # Eine Zelle nach rechts
                                arbeit_flexion_links = sub_row[4] if len(sub_row) > 4 else None  # Drei Zellen nach rechts

                    # Debugging-Ausgabe: Überprüfe die gefundenen Werte
                    print(f"Debug - Datei: {file}")
                    print(f"Extension Rechts: {arbeit_extension_rechts}, Extension Links: {arbeit_extension_links}")
                    print(f"Flexion Rechts: {arbeit_flexion_rechts}, Flexion Links: {arbeit_flexion_links}")

            # Ergebnisse speichern
            results.append({
                "Datei": file,
                "Arbeit Extension Rechts": arbeit_extension_rechts,
                "Arbeit Extension Links": arbeit_extension_links,
                "Arbeit Flexion Rechts": arbeit_flexion_rechts,
                "Arbeit Flexion Links": arbeit_flexion_links
            })

        except Exception as e:
            print(f"Fehler bei Datei: {file}. Fehler: {e}")
        finally:
            workbook.close()

# Ergebnisse als DataFrame speichern
results_df = pd.DataFrame(results)

# Ergebnisse in Excel speichern
output_path = os.path.join(folder_path, "Ergebnisse.xlsx")
results_df.to_excel(output_path, index=False)

print(f"Ergebnisse gespeichert unter: {output_path}")


Debug - Datei: 141.xlsx
Extension Rechts: 289, Extension Links: 245
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 179.xlsx
Extension Rechts: 170, Extension Links: 218
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 188.xlsx
Extension Rechts: 243, Extension Links: 251
Flexion Rechts: 164, Flexion Links: 199
Debug - Datei: 193.xlsx
Extension Rechts: 152, Extension Links: 152
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 200.xlsx
Extension Rechts: 217, Extension Links: 203
Flexion Rechts: 138, Flexion Links: 146
Debug - Datei: 203.xlsx
Extension Rechts: 209, Extension Links: 191
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 209.xlsx
Extension Rechts: 159, Extension Links: 113
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 210.xlsx
Extension Rechts: 171, Extension Links: 218
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 225.xlsx
Extension Rechts: 229, Extension Links: 245
Flexion Rechts: None, Flexion Links: None
Debug - Datei:

In [7]:
import os
import pandas as pd
from openpyxl import load_workbook

# Pfad zum Ordner mit den Excel-Dateien
folder_path = r"K:\Team\Böhmer_Michael\TSA\ML\Basistabelle\Work_ergänzen"

# Ergebnisse speichern
results = []

# Alle Excel-Dateien im Ordner durchlaufen
for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        try:
            # Excel-Datei laden
            workbook = load_workbook(file_path, read_only=True)
            matching_sheets = []

            # Überprüfung aller Arbeitsblätter
            for sheet_name in workbook.sheetnames:
                sheet = workbook[sheet_name]
                if any(cell and cell[0] == "Isokinetisch Kon/Kon" for cell in sheet.iter_rows(min_col=1, max_col=1, values_only=True)):
                    matching_sheets.append(sheet_name)

            if len(matching_sheets) > 1:
                print(f"Mehrere passende Arbeitsblätter in Datei: {file}. Datei wird übersprungen.")
                continue
            elif len(matching_sheets) == 0:
                print(f"Kein passendes Arbeitsblatt in Datei: {file}. Datei wird übersprungen.")
                continue

            sheet = workbook[matching_sheets[0]]
            arbeit_extension_rechts = arbeit_extension_links = None
            arbeit_flexion_rechts = arbeit_flexion_links = None

            for row_idx, row in enumerate(sheet.iter_rows(values_only=True), start=1):
                if row and "Isokinetisch Kon/Kon" in row:
                    # Position der Startzelle "Isokinetisch Kon/Kon"
                    col_idx = row.index("Isokinetisch Kon/Kon") + 1

                    # 1. Arbeit Extension Rechts
                    if sheet.cell(row=row_idx, column=col_idx + 1).value == "Extensoren (Kon)":
                        arbeit_extension_rechts = sheet.cell(row=row_idx + 7, column=col_idx + 1).value
                        arbeit_extension_links = sheet.cell(row=row_idx + 8, column=col_idx + 1).value

                    # 2. Arbeit Flexion Rechts und Links
                    if sheet.cell(row=row_idx + 1, column=col_idx + 4).value == "Wert":
                        arbeit_flexion_rechts = sheet.cell(row=row_idx + 7, column=col_idx + 4).value
                        arbeit_flexion_links = sheet.cell(row=row_idx + 8, column=col_idx + 4).value

                    # Debugging-Ausgabe: Überprüfe die gefundenen Werte
                    print(f"Debug - Datei: {file}")
                    print(f"Extension Rechts: {arbeit_extension_rechts}, Extension Links: {arbeit_extension_links}")
                    print(f"Flexion Rechts: {arbeit_flexion_rechts}, Flexion Links: {arbeit_flexion_links}")
                    break  # Beende die Suche, wenn die Zelle gefunden wurde

            # Ergebnisse speichern
            results.append({
                "Datei": file,
                "Arbeit Extension Rechts": arbeit_extension_rechts,
                "Arbeit Extension Links": arbeit_extension_links,
                "Arbeit Flexion Rechts": arbeit_flexion_rechts,
                "Arbeit Flexion Links": arbeit_flexion_links
            })

        except Exception as e:
            print(f"Fehler bei Datei: {file}. Fehler: {e}")
        finally:
            workbook.close()

# Ergebnisse als DataFrame speichern
results_df = pd.DataFrame(results)

# Ergebnisse in Excel speichern
output_path = os.path.join(folder_path, "Ergebnisse.xlsx")
results_df.to_excel(output_path, index=False)

print(f"Ergebnisse gespeichert unter: {output_path}")


Debug - Datei: 141.xlsx
Extension Rechts: 289, Extension Links: 245
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 179.xlsx
Extension Rechts: 170, Extension Links: 218
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 188.xlsx
Extension Rechts: 243, Extension Links: 251
Flexion Rechts: 164, Flexion Links: 199
Debug - Datei: 193.xlsx
Extension Rechts: 152, Extension Links: 152
Flexion Rechts: 89, Flexion Links: 83
Debug - Datei: 200.xlsx
Extension Rechts: 217, Extension Links: 203
Flexion Rechts: 138, Flexion Links: 146
Debug - Datei: 203.xlsx
Extension Rechts: 209, Extension Links: 191
Flexion Rechts: 149, Flexion Links: 156
Debug - Datei: 209.xlsx
Extension Rechts: 159, Extension Links: 113
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 210.xlsx
Extension Rechts: 171, Extension Links: 218
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 225.xlsx
Extension Rechts: 229, Extension Links: 245
Flexion Rechts: None, Flexion Links: None
Debug - Datei: 227.x